<a href="https://colab.research.google.com/github/Igor-ID/g.tec-Hackathon-Unresponsive-Wakefullness-Syndrom-Data-Analysis/blob/main/gtec_hackathon_uws_p300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline
!pip install mne
import mne
import matplotlib.pyplot as plt
import scipy
import os
from scipy.io import loadmat
import numpy as np
import mne
import pandas as pd
from mne.decoding import Vectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from google.colab import drive
drive.mount('/content/drive')



     |████████████████████████████████| 7.5 MB 23.0 MB/s 
Mounted at /content/drive


Helper func to create epochs object

In [7]:
# file_name = "/content/drive/My Drive/data/P2_high1.mat"
# # file_name2 = "data/P1_high2.mat"
# data = loadmat(file_name)
# # data2 = loadmat(file_name2)
# eeg_data = data['y']
# eeg_data.shape
# helper func to create epoch object
def create_epochs(starting_dir="/content/drive/My Drive/data/", item='P1_high1.mat'):
    eeg_ch = ['Fz', 'C3', 'Cz', 'C4', 'CP1', 'CPz', 'CP2', 'Pz']
    fs = 256
    n_channels = 8
    event_id = dict(distractor=-1, rest=0, nontarget=1, target=2)
    info = mne.create_info(ch_names=eeg_ch, sfreq=fs, ch_types='eeg')
    info.set_montage('standard_1020')
    file_name = os.path.join(starting_dir, item)
    data = loadmat(file_name)
    eeg_data = data['y']

    raw = mne.io.RawArray(eeg_data.T, info)
    stim_data = data['trig']
    stim_data[stim_data == -1] = 3  # change stimulus negative value of -1 to 3 for correct stimulus reading by mne
    info_event = mne.create_info(['STI'], sfreq=fs, ch_types='stim')
    stim_raw = mne.io.RawArray(stim_data.T, info_event)
    raw.add_channels([stim_raw], force_update_info=True)

    events = mne.find_events(raw, stim_channel='STI')

    raw.notch_filter(freqs=50)  # creating band-stop filter to eliminate power line noise (US - 60Hz, EU - 50Hz)
    raw.filter(l_freq=0.1, h_freq=30)  # creating band-pass filter, to eliminate non-physiological frequencies

    epochs = mne.Epochs(raw, events, tmin=-0.1, tmax=0.6, preload=True)
    return epochs

Can we predict trial type from EEG activity?

In [8]:
epochs_train = create_epochs()
epochs_train.pick_types(eeg=True)
# event_ids = {'distractor': 3, 'rest': 0, 'nontarget': 1, 'target': 2}
X = epochs_train.get_data()  # features
y = epochs_train.events[:, -1]  # labels
X, y

Creating RawArray with float64 data, n_channels=8, n_times=38976
    Range : 0 ... 38975 =      0.000 ...   152.246 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=38976
    Range : 0 ... 38975 =      0.000 ...   152.246 secs
Ready.
480 events found
Event IDs: [1 2 3]
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1691 samples (6.605 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpas

(array([[[-17.39263534, -19.19282209, -21.44507722, ...,   5.34220556,
           -0.25624422,  -4.81489551],
         [-10.33766136, -12.08048818, -14.19101489, ...,   3.83773912,
            0.4088002 ,  -2.72862412],
         [-15.03758017, -16.86901221, -18.74485438, ...,  10.10169478,
            4.7204059 ,  -0.14130736],
         ...,
         [-12.61412642, -14.59197742, -16.31698682, ...,   4.52273181,
           -1.19877254,  -6.40834764],
         [-11.44505764, -13.29306011, -15.29070655, ...,  16.07607512,
           10.17461714,   4.31416866],
         [ -8.68670464, -10.89000821, -12.98459907, ...,  10.58791633,
            4.85713451,  -0.78372032]],
 
        [[-23.69125524, -25.95330725, -28.42228384, ...,  23.40558716,
           25.39678315,  28.48444067],
         [ -7.26549756,  -9.73585104, -11.55358044, ...,  21.8698838 ,
           22.77407955,  24.55957299],
         [-25.87945667, -26.75124647, -27.3989142 , ...,  13.03378935,
           14.47135197,  17.3259

In [9]:
# shape
X.shape, y.shape

((480, 8, 181), (480,))

In [10]:
epochs_test = create_epochs(item='P1_high2.mat')
epochs_test.pick_types(eeg=True)
X_test = epochs_test.get_data()  # features
y_test = epochs_test.events[:, -1]  # labels
X_test.shape, y_test.shape

Creating RawArray with float64 data, n_channels=8, n_times=39000
    Range : 0 ... 38999 =      0.000 ...   152.340 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=39000
    Range : 0 ... 38999 =      0.000 ...   152.340 secs
Ready.
480 events found
Event IDs: [1 2 3]
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1691 samples (6.605 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpas

((480, 9, 181), (480,))

X has the wrong shape - `samples`, `channels`, `times`, should be `samples`, `features`.

We can use `mne.decoding.Vectorizer` to correctly shape the data. It fits right into a scikit-learn pipeline.

In [12]:
clf = make_pipeline(Vectorizer(), StandardScaler(), LinearDiscriminantAnalysis())
clf

Pipeline(steps=[('vectorizer',
                 <mne.decoding.transformer.Vectorizer object at 0x7f587e1177d0>),
                ('standardscaler', StandardScaler()),
                ('lineardiscriminantanalysis', LinearDiscriminantAnalysis())])

The resulting object behaves exactly as any other scikit-learn classifier:

In [13]:
clf.fit(X, y)

Pipeline(steps=[('vectorizer',
                 <mne.decoding.transformer.Vectorizer object at 0x7f587e1177d0>),
                ('standardscaler', StandardScaler()),
                ('lineardiscriminantanalysis', LinearDiscriminantAnalysis())])

In [14]:
y_test[:9]

array([3, 3, 3, 1, 3, 3, 3, 2, 1])

In [18]:
# we have slightly different shape of the initial data, improvements required
clf.predict(X_test[:9])

ValueError: ignored

Of course, usually we care about cross-validated scores.

In [20]:
cross_val_score(clf, X, y, cv=10)  # accuracy

array([0.625     , 0.33333333, 0.375     , 0.39583333, 0.25      ,
       0.45833333, 0.66666667, 0.33333333, 0.27083333, 0.47916667])